# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import os
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

We can import the S&P 500 symbols using the script named 'tickers.py'. This function scrapes the Wikipedia web page to retrieve the S&P 500 symbols, ensuring that the list is updated.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
from tickers import get_tickers
stocks= get_tickers()

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.


In [4]:
from api_key import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud.

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
response = requests.get(url=api_url)
response.status_code

200

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy.

With that said, the data isn't in a proper format yet. We need to parse it first.

In [16]:
 data = response.json()
 data_dict = dict(ticker= [data[0]['symbol']],
    price = [data[0]['latestPrice']],
    market_cap = [data[0]['marketCap']],
    shares_to_buy = ['N/A'])
 data_dict

{'ticker': ['AAPL'],
 'price': [176.55],
 'market_cap': [2726264090550],
 'shares_to_buy': ['N/A']}

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
df = pd.DataFrame(data_dict)
df

,ticker,price,market_cap,shares_to_buy
0,AAPL,176.55,2726264090550,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 2000 tickers per request.

In [8]:
def chunk (lst, n):
    for i in range(0,len(lst),n):
      yield lst[i:i+n]


def get_stocks_data_2(stock_list):
    symbol_groups= list(chunk(stocks['Ticker'],2000))
    symbol_strings = []
    for i in range(0,len(symbol_groups)):
      symbol_strings.append(','.join(symbol_groups[i]))

    for symbol_string in symbol_strings:
        api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()

    all_data = []
    for i in data:
        dict_data= dict(ticker = i['symbol'],
        price = i['latestPrice'],
        market_cap = i['marketCap'],
        shares_to_buy= 'N/A')
        all_data.append(dict_data)

    df= pd.DataFrame(all_data)
    return df

In [9]:
sp500 = get_stocks_data_2(stocks['Ticker'])
sp500

,ticker,price,market_cap,shares_to_buy
0,MMM,91.31,50527416377,N/A
1,AOS,86.50,13215083086,N/A
2,ABT,109.11,189325957773,N/A
3,ABBV,162.28,287340592401,N/A
4,ACN,315.40,211451017427,N/A
...,...,...,...,...
498,XYL,128.11,31053672860,N/A
499,YUM,137.80,38790700000,N/A
500,ZBRA,289.68,14888836780,N/A
501,ZBH,124.48,25575841420,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [10]:
portfolio_size = input('Enter the value of your portfolio: ')

while True:
    try:
      val = float(portfolio_size)
      break
    except ValueError:
      print('Please enter a number')
      portfolio_size = input('Enter the value of your portfolio: ')

Enter the value of your portfolio: 1000000


In [11]:
position_size = val/len(sp500)
for i in range(0,len(sp500)):
  sp500.loc[i, 'shares_to_buy'] = math.floor(position_size/sp500.loc[i, 'price'])

sp500

,ticker,price,market_cap,shares_to_buy
0,MMM,91.31,50527416377,21
1,AOS,86.50,13215083086,22
2,ABT,109.11,189325957773,18
3,ABBV,162.28,287340592401,12
4,ACN,315.40,211451017427,6
...,...,...,...,...
498,XYL,128.11,31053672860,15
499,YUM,137.80,38790700000,14
500,ZBRA,289.68,14888836780,6
501,ZBH,124.48,25575841420,15


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [12]:
writer = pd.ExcelWriter('Recommended Trades.xlsx', engine= 'xlsxwriter')
sp500.to_excel(writer, sheet_name= 'Recommended Trades', index= False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [13]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format({'font_color':font_color,
                                        'bg_color':background_color,
                                        'border': 1})

dollar_format = writer.book.add_format({'num_format':'$0.00',
                                        'font_color':font_color,
                                        'bg_color':background_color,
                                        'border': 1})

integer_format = writer.book.add_format({'num_format':'0',
                                        'font_color':font_color,
                                        'bg_color':background_color,
                                        'border': 1})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

Let's simplify this by putting it in 2 loops:

In [14]:
column_formats = {
    'A': ['Ticker',string_format],
    'B': ['Share Price',dollar_format],
    'C': ['Market Cap',dollar_format],
    'D': ['Shares to buy',integer_format]
    }
for column in column_formats.keys():
  writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',18,column_formats[column][1])
  writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0],string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [15]:
writer.close()